In [1]:
import numpy as np
import importlib
import operator
import util as uti
import matplotlib.pyplot as plt
from collections import OrderedDict
importlib.reload(uti)

# import math lib
import math

# import Qiskit
from qiskit import Aer, IBMQ, execute, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import GroverOperator
from qiskit_machine_learning.circuit.library import RawFeatureVector

# import basic plot tools
#import qiskit.visualization as vis
from qiskit.visualization import plot_histogram
%matplotlib inline

import sys
import os
sys.path.append(os.path.abspath('../quantum'))
import oracles as ora
import grover as gro
importlib.reload(gro)

/home/kinga/.local/lib/python3.6/site-packages/qiskit/__init__.py:66: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of qiskit-terra 0.17.0. Support for running Qiskit with Python 3.6 will be removed in qiskit-terra 0.20.0.
  DeprecationWarning,


<module 'grover' from '/home/kinga/mnt6/laspaclu/quantum/grover.py'>

In [12]:
def measure_by_prob(counts, n):
    # sort indices by prob
    sorted_counts = OrderedDict(sorted(counts.items(), key=lambda x: x[1], reverse=True))
    print(sorted_counts)
    # take most probable value (if it is in the limits of array)
    for k in sorted_counts.keys():
        if int(k,2) < n:
            return int(k,2)
    print('no valid index found')
    return -1

In [13]:
def duerr_hoyer_minimization(distances):
    
    nn = int(math.floor(math.log2(len(distances)) + 1))
    iter_n = math.ceil(np.sqrt(2**nn)) # suggested in duerr & hoyer 
    
    threshold_oracles = ora.create_threshold_oracle_set(distances)
    
    # init first random threshold
    idx = np.random.randint(0, len(distances)-1)
    
    # iterate
    for _ in range(iter_n):
        
        # pick next threshold
        threshold = distances[idx]
        marked_n = len(ora.get_indices_to_mark(distances, threshold))
        
        # create oracle combi and grover algo
        oracle_qc = ora.create_oracle_lincombi(threshold, distances, threshold_oracles)
        grover_qc = gro.grover_circuit(nn, oracle_qc, marked_n)
        
        # apply grover algo (only one shot to get a true collapsed value)
        simulator = Aer.get_backend('qasm_simulator')
        counts = execute(grover_qc, backend=simulator, shots=1000).result().get_counts(grover_qc)
        idx = measure_by_prob(counts, len(distances))
        print('next minimum guess: dist[{}] = {}'.format(idx, distances[idx]))
        
    print('final minimum found dist[{}] = {}'.format(idx, distances[idx]))
    return distances[idx]

In [14]:
distances = np.array([17,4,6,2,8,32,7,16])

In [15]:
duerr_hoyer_minimization(distances)

OrderedDict([('0001', 273), ('0110', 251), ('0010', 239), ('0011', 237)])
next minimum guess: dist[1] = 4
OrderedDict([('0011', 966), ('1000', 9), ('0001', 4), ('1101', 4), ('0101', 4), ('0111', 3), ('1011', 3), ('0110', 2), ('1111', 2), ('1110', 1), ('1010', 1), ('1100', 1)])
next minimum guess: dist[3] = 2
OrderedDict([('1110', 75), ('1001', 71), ('0000', 69), ('0100', 66), ('1011', 66), ('0011', 64), ('1111', 63), ('0111', 63), ('0010', 62), ('1010', 62), ('0101', 62), ('1100', 58), ('1101', 57), ('0110', 57), ('1000', 56), ('0001', 49)])
next minimum guess: dist[0] = 17
OrderedDict([('0110', 161), ('0001', 143), ('0011', 142), ('0100', 139), ('0111', 137), ('0010', 130), ('1001', 21), ('1000', 21), ('0101', 20), ('0000', 16), ('1011', 15), ('1111', 12), ('1110', 12), ('1010', 11), ('1101', 10), ('1100', 10)])
next minimum guess: dist[6] = 7
final minimum found dist[6] = 7


7

In [81]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}

In [82]:
sorted_x = OrderedDict(sorted(x.items(), key=lambda xx: xx[1]))

In [83]:
sorted_x

OrderedDict([(0, 0), (2, 1), (1, 2), (4, 3), (3, 4)])